Download Schema

In [3]:
import requests
import os
import pandas as pd

# Set up the logger
import logging

logger = logging.getLogger()

def download(url, filename=None):
  """Downloads a file from a URL and saves it locally.

  Args:
    url: The URL of the file to download.
    filename: The filename to save the downloaded file as. If not specified,
      the filename will be extracted from the URL.
  """
  response = requests.get(url, stream=True)
  response.raise_for_status()
  
  if filename is None:
    filename = url.split("/")[-1]

  if os.path.isfile(filename):
    os.remove(filename)  

  if os.path.isfile(filename+".wal"):
    os.remove(filename+".wal")  

  with open(filename, "wb") as f:
    for chunk in response.iter_content(1024):
      if chunk:  # filter out keep-alive new chunks
        f.write(chunk)
        
  for x in os.listdir():
    if x.endswith(".db") or x.endswith(".wal"):
        # Prints only text file present in My Folder
        print(x)
        
  return filename

In [1]:
import os
database_name = "finance"
filename = f"/tmp/{database_name}.db"
if not os.path.isfile(filename):
    url = f"http://test-public-storage-440955376164.s3-website.us-east-1.amazonaws.com/catalogs/{database_name}.db"
    filename = download(url, filename)    

Open Database and Login

In [2]:
import vaultdb
connection = vaultdb.login.cognito("vaultdb", "test123", filename, aws_region="us-east-1")

ModuleNotFoundError: No module named 'vaultdb'

In [3]:
connection.execute("SELECT * FROM vaultdb_configs").fetchdf()

database_name  database_oid schema_name  schema_oid          config_name  \
0          test          1102    security        1107     application_name   
1          test          1102    security        1107     identity_pool_id   
2          test          1102    security        1107               remote   
3          test          1102    security        1107    remote_merge_path   
4          test          1102    security        1107  user_pool_client_id   
5          test          1102    security        1107         user_pool_id   

   config_oid  internal  temporary  \
0        1148     False      False   
1        1154     False      False   
2        1160     False      False   
3        1166     False      False   
4        1172     False      False   
5        1178     False      False   

                                     config_value  
0                                            test  
1  us-east-1:3aa03da0-b11e-444d-ad5d-ec7e4844819f  
2         s3://test-data-440955376164/merged_data  
3           s3://test-public-storage-440955376164  
4                      2ckshe5jnhi3cc2akodabuigct  
5                             us-east-1_tKxja4leu

In [4]:
connection.execute("select * from duckdb_secrets();").fetch_df()

name type provider  persistent storage                    scope  \
0  vaultdb   s3   config       False  memory  [s3://, s3n://, s3a://]   

                                       secret_string  
0  name=vaultdb;type=s3;provider=config;serializa...

In [5]:
connection.execute("select * from information_schema.tables").fetchdf()

table_catalog table_schema        table_name  table_type  \
0          test         main         another_T  BASE TABLE   
1          test         main             demo2  BASE TABLE   
2          test         main             quote  BASE TABLE   
3          test         main  tbl_ProductSales  BASE TABLE   
4          test         main           tickers  BASE TABLE   

  self_referencing_column_name reference_generation user_defined_type_catalog  \
0                         None                 None                      None   
1                         None                 None                      None   
2                         None                 None                      None   
3                         None                 None                      None   
4                         None                 None                      None   

  user_defined_type_schema user_defined_type_name is_insertable_into is_typed  \
0                     None                   None                YES       NO   
1                     None                   None                YES       NO   
2                     None                   None                YES       NO   
3                     None                   None                YES       NO   
4                     None                   None                YES       NO   

  commit_action TABLE_COMMENT  
0          None          None  
1          None          None  
2          None          None  
3          None          None  
4          None          None

In [6]:
connection.execute("SET timezone='America/New_York';")
connection.execute("select * from quote where date='2024-06-24'").fetchdf()

Empty DataFrame
Columns: [date, open, high, low, close, volume, dividends, stock_splits, symbol]
Index: []

Data Inheritance

In [7]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM tickers").fetchdf()

exchange  symbol                                               name  \
0      nasdaq    SRCL                       Stericycle Inc. Common Stock   
1      nasdaq    SRDX                        Surmodics Inc. Common Stock   
2      nasdaq     SRM                SRM Entertainment Inc. Common Stock   
3      nasdaq    SRPT        Sarepta Therapeutics Inc. Common Stock (DE)   
4      nasdaq    SRRK      Scholar Rock Holding Corporation Common Stock   
...       ...     ...                                                ...   
7049     amex    YCBD                            cbdMD Inc. Common Stock   
7050     amex  YCBD^A  cbdMD Inc. 8.0% Series A Cumulative Convertibl...   
7051     amex    ZDGE                   Zedge Inc. Class B Common Stock    
7052     amex     ZOM                       Zomedica Corp. Common Shares   
7053     amex    ZONE      CleanCore Solutions Inc. Class B Common Stock   

     lastsale  netchange  pctchange     marketCap        country  ipoyear  \
0       57.94    -1.2800     -0.987  3.744751e+10  United States   1999.0   
1       41.96     0.6250      1.571  7.255109e+09  United States   2016.0   
2        1.16    -0.0100     -0.094  0.000000e+00  United States   2023.0   
3      158.15    -0.0450     -0.451  3.042648e+08  United States   2020.0   
4       8.355    -3.2000     -5.053  3.585089e+09  United States      NaN   
...       ...        ...        ...           ...            ...      ...   
7049     0.58    -0.0100     -1.695  2.244068e+06  United States   2017.0   
7050     0.57    -0.0500     -8.065           NaN  United States      NaN   
7051      3.0    -0.0500     -1.639  4.341252e+07                  2016.0   
7052   0.1476     0.0014      0.958  1.446406e+08  United States   2017.0   
7053     2.55     0.2500     10.870  2.028759e+07                  2024.0   

       volume                  sector  \
0      605488             Industrials   
1     1011023             Health Care   
2        2806  Consumer Discretionary   
3      375220             Health Care   
4      517096             Health Care   
...       ...                     ...   
7049     6520  Consumer Discretionary   
7050     2006                           
7051    77696              Technology   
7052  1320954             Health Care   
7053     6826             Industrials   

                                               industry  \
0                                Environmental Services   
1                            Medical/Dental Instruments   
2                      Recreational Games/Products/Toys   
3            Biotechnology: Pharmaceutical Preparations   
4     Biotechnology: Biological Products (No Diagnos...   
...                                                 ...   
7049                            Package Goods/Cosmetics   
7050                                                      
7051            Computer Software: Prepackaged Software   
7052         Biotechnology: Pharmaceutical Preparations   
7053                    Industrial Machinery/Components   

                                 url  
0       /market-activity/stocks/srcl  
1       /market-activity/stocks/srdx  
2        /market-activity/stocks/srm  
3       /market-activity/stocks/srpt  
4       /market-activity/stocks/srrk  
...                              ...  
7049    /market-activity/stocks/ycbd  
7050  /market-activity/stocks/ycbd^a  
7051    /market-activity/stocks/zdge  
7052     /market-activity/stocks/zom  
7053    /market-activity/stocks/zone  

[7054 rows x 13 columns]

Insert Data Locally

In [8]:
connection.execute("INSERT INTO another_T VALUES (81,92,93,94,95,96,97,98)")

View Local Data

In [10]:
connection.execute(f"PRAGMA disable_data_inheritance;")
connection.execute("SELECT * FROM quote").fetchdf()

Empty DataFrame
Columns: [date, open, high, low, close, volume, dividends, stock_splits, symbol]
Index: []

Enable Inheritance

In [13]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

col1  col2  col3  col4  col5  col6  col7  col8
0    81    92    93    94    95    96    97    98

Push local Data

In [14]:
connection.execute('PUSH DATABASE dev;').fetchdf()

Empty DataFrame
Columns: [Success]
Index: []

Truncate Local data

In [9]:
connection.execute('TRUNCATE DATABASE test;').fetchdf()

Empty DataFrame
Columns: [Success]
Index: []

View remote Data

In [16]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

col1  col2  col3  col4  col5  col6  col7  col8
0    81    92    93    94    95    96    97    98

Create Objects Locally

In [17]:
connection.execute('BEGIN TRANSACTION;')
connection.execute('CREATE TABLE demo2 (col1 INT, col2 INT, col3 INT, col4 INT, col5 INT, col6 INT, col7 INT, col8 INT)')
connection.execute("INSERT INTO demo2 VALUES (1,2,3,4,5,6,7,8), (11,22,33,44,55,66,77,88), (111,222,333,444,555,666,777,888), (1111,2222,3333,4444,5555,6666,7777,8888)")
connection.execute('COMMIT;')

Push Schema Changes on to server

In [18]:
connection.execute('PUSH DATABASE dev;').fetchdf()

Empty DataFrame
Columns: [Success]
Index: []